# Module 1. Check Inference Results & Local Mode Deployment
---

## Overview

본 핸즈온은 AWS AIML Blog의 내용을 기반으로 MNIST 예제 대신 좀 더 실용적인 한국어 자연어 처리 예시를 다루며, 총 3종류(Sentiment Classification, KorSTS, KoBART)의 자연어 처리 모델을 SageMaker 다중 컨테이너 엔드포인트(Multi-container endpoint)로 배포하는 법을 익혀 봅니다.

이미 SageMaker 기본 개념(로컬 모드, 호스팅 엔드포인트)과 자연어 처리 & Huggingface을 다뤄 보신 분들은 이 섹션을 건너 뛰고 다음 노트북으로 진행하셔도 됩니다.

### References
- AWS AIML Blog: https://aws.amazon.com/ko/blogs/machine-learning/deploy-multiple-serving-containers-on-a-single-instance-using-amazon-sagemaker-multi-container-endpoints/
- Developer Guide: https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html

In [20]:
!pip install -qU sagemaker botocore boto3 awscli
!pip install --ignore-installed PyYAML
!pip install transformers==4.12.5

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.22.20 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [ ]:
import json
import os
import sys
import torch
import boto3
import sagemaker
import datetime
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from src.utils import print_outputs, prepare_model_artifact, NLPPredictor 

role = get_execution_role()
boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_client = boto_session.client("sagemaker")
sm_runtime = boto_session.client("sagemaker-runtime")

<br>

## 1. Check Inference Results & Debugging
---

로컬 엔드포인트나 호스팅 엔드포인트 배포 전, 로컬 환경 상에서 직접 추론을 수행하여 결과를 확인합니다. 참고로, SageMaker에서 TensorFlow를 제외한 머신 러닝 프레임워크 추론 컨테이너는 아래의 인터페이스를 사용합니다.

#### Option 1.
- `model_fn(model_dir)`: 네트워크 아키텍처를 정의하고 S3의 model_dir에 저장된 모델 아티팩트를 로드합니다.
- `input_fn(request_body, content_type)`: 입력 데이터를 전처리합니다. (예: request_body로 전송된 bytearray 배열을 PIL.Image로 변환 수 cropping, resizing, normalization등의 전처리 수행). content_type은 입력 데이터 종류에 따라 다양하게 처리 가능합니다. (예: application/x-npy, application/json, application/csv 등)
- `predict_fn(input_object, model)`: input_fn을 통해 들어온 데이터에 대해 추론을 수행합니다.
- `output_fn(prediction, accept_type)`: predict_fn에서 받은 추론 결과를 추가 변환을 거쳐 프론트 엔드로 전송합니다.

#### Option 2.
- `model_fn(model_dir)`: 네트워크 아키텍처를 정의하고 S3의 model_dir에 저장된 모델 아티팩트를 로드합니다.
- `transform_fn(model, request_body, content_type, accept_type)`: input_fn(), predict_fn(), output_fn()을 transform_fn()으로 통합할 수 있습니다.

모델, 배포에 초점을 맞추기 위해 Huggingface에 등록된 `KoELECTRA-Small-v3` 모델을 기반으로 네이버 영화 리뷰 데이터셋과 KorSTS (Korean Semantic Textual Similarity) 데이터셋으로 파인 튜닝하였습니다. 파인 튜닝은 온프레미스나 Huggingface on SageMaker로 쉽게 수행 가능합니다. 

- KoELECTRA: https://github.com/monologg/KoELECTRA
- Huggingface on Amazon SageMaker: https://huggingface.co/docs/sagemaker/main


### Model A: Sentiment Classification

네이버 영화 리뷰 데이터의 긍정/부정 판별 예시입니다. 
- Naver sentiment movie corpus: https://github.com/e9t/nsmc
- 예시
    - '이 영화는 최고의 영화입니다' => {"predicted_label": "Pos", "score": 0.96}
    - '최악이에요. 배우의 연기력도 좋지 않고 내용도 너무 허접합니다' => {"predicted_label": "Neg", "score": 0.99}

In [2]:
!pygmentize src/inference_nsmc.py

import json
import sys
import logging
import torch
from torch import nn
from transformers import ElectraConfig
from transformers import ElectraModel, AutoTokenizer, ElectraTokenizer, ElectraForSequenceClassification

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(filename='tmp.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

max_seq_length = 128
classes = ['Neg', 'Pos']

tokenizer = AutoTokenizer.from_pretrained("daekeun-ml/koelectra-small-v3-nsmc")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def model_fn(model_path=None):
    ####
    # If you have your own trained model
    # Huggingface pre-trained model: 'monologg/koelectra-small-v3-discriminator'
    ####
    #config = ElectraConfig.from_json_file(f'{model_path}/config.json')
    #model = ElectraForSequenceClassification.from_pretrained(f'{model_pat

In [3]:
from src.inference_nsmc import model_fn, input_fn, predict_fn, output_fn
modelA_path = 'model-nsmc'

with open('samples/nsmc.txt', mode='rb') as file:
    modelA_input_data = file.read()

modelA = model_fn(modelA_path)
transformed_inputs = input_fn(modelA_input_data)
predicted_classes_jsonlines = predict_fn(transformed_inputs, modelA)
modelA_outputs = output_fn(predicted_classes_jsonlines)
print(modelA_outputs[0])    

Downloading:   0%|          | 0.00/414 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/521k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


[{inference_nsmc.py:47} INFO - input text: 이 영화는 최고의 영화입니다
[{inference_nsmc.py:47} INFO - input text: 최악이에요. 배우의 연기력도 좋지 않고 내용도 너무 허접합니다
[{inference_nsmc.py:77} INFO - predicted_class: Pos
[{inference_nsmc.py:84} INFO - jsonline: {"predicted_label": "Pos", "score": 0.9619030952453613}
[{inference_nsmc.py:77} INFO - predicted_class: Neg
[{inference_nsmc.py:84} INFO - jsonline: {"predicted_label": "Neg", "score": 0.9994170665740967}
{"predicted_label": "Pos", "score": 0.9619030952453613}
{"predicted_label": "Neg", "score": 0.9994170665740967}


### Model B: Semantic Textual Similarity (STS)

두 문장간의 유사도를 정량화하는 예시입니다.
- KorNLI and KorSTS: https://github.com/kakaobrain/KorNLUDatasets
- 예시
    - ['맛있는 라면을 먹고 싶어요', '후루룩 쩝쩝 후루룩 쩝쩝 맛좋은 라면'] => {"score": 4.78}
    - ['뽀로로는 내친구', '머신러닝은 러닝머신이 아닙니다.'] => {"score": 0.23}

In [4]:
!pygmentize src/inference_korsts.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
import json
import sys
import logging
import torch
from torch import nn
from transformers import ElectraConfig
from transformers import ElectraModel, AutoTokenizer, ElectraTokenizer, ElectraForSequenceClassification

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(filename='tmp.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

max_seq_length = 128
tokenizer = AutoTokenizer.from_pretrained("daekeun-ml/koelectra-small-v3-korsts")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Huggingface pre

In [5]:
from src.inference_korsts import model_fn, input_fn, predict_fn, output_fn
modelB_path = 'model-korsts'

with open('samples/korsts.txt', mode='rb') as file:
    modelB_input_data = file.read()    
    
modelB = model_fn(modelB_path)
transformed_inputs = input_fn(modelB_input_data)
predicted_classes_jsonlines = predict_fn(transformed_inputs, modelB)
modelB_outputs = output_fn(predicted_classes_jsonlines)
print(modelB_outputs[0])

Downloading:   0%|          | 0.00/416 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/521k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/885 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.0M [00:00<?, ?B/s]

[{inference_korsts.py:44} INFO - input text: ['맛있는 라면을 먹고 싶어요', '후루룩 쩝쩝 후루룩 쩝쩝 맛좋은 라면']
[{inference_korsts.py:44} INFO - input text: ['뽀로로는 내친구', '머신러닝은 러닝머신이 아닙니다.']
[{inference_korsts.py:71} INFO - jsonline: {"score": 4.786738872528076}
[{inference_korsts.py:71} INFO - jsonline: {"score": 0.2319067120552063}
{"score": 4.786738872528076}
{"score": 0.2319067120552063}


### Model C: KoBART (Korean Bidirectional and Auto-Regressive Transformers)

문서 내용(예: 뉴스 기사)을 요약하는 예시입니다.

- KoBART: https://github.com/SKT-AI/KoBART
- KoBART Summarization: https://github.com/seujung/KoBART-summarization

In [6]:
!pygmentize src/inference_kobart.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
import json
import sys
import logging
import torch
from torch import nn
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(filename='tmp.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def model_fn(model_path=None):
    model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")
 

S3로 모델 아티팩트를 복사하는 대신 Huggingface에 등록된 모델을 그대로 사용합니다. model.pth는 0바이트의 빈 파일이며, 추론을 수행하기 위한 소스 코드들만 아카이빙됩니다.

In [7]:
from src.inference_kobart import model_fn, transform_fn
modelC_path = 'model-kobart'
f = open(f"{modelC_path}/model.pth", 'w')
f.close()

with open('samples/kobart.txt', mode='rb') as file:
    modelC_input_data = file.read()

modelC = model_fn('./')
outputs = transform_fn(modelC, modelC_input_data)

with open('samples/kobart.txt', mode='rb') as file:
    modelC_input_data = file.read()

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/302 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

[{inference_kobart.py:36} INFO - input text: AWS가 10일 AWS 리인벤트를 통해 머신러닝 서비스인 아마존 세이지메이커(Amazon SageMaker)의 9가지 새로운 기능을 발표했다. 아마존 세이지메이커 데이터 랭글러(Amazon SageMaker Data Wrangler)를 비롯해 아마존 세이지메이커 피처 스토어 등 다양한 기술들이 속속 베일을 벗었다. 스와미 시바수브라마니안(Swami Sivasubramanian), AWS 아마존 머신러닝 부사장은 '수십만 명의 일반 개발자와 데이터 과학자가 업계 최고의 머신러닝 서비스인 아마존 세이지메이커를 활용해 맞춤형 머신러닝 모델 제작, 훈련 및 배치에 대한 장벽을 제거했다'면서 '개발자가 더 나은 가시성, 설명 가능성 및 자동화를 대규모로 구현하는 맞춤형 머신러닝 모델을 준비, 제작, 훈련, 설명, 검사, 모니터링, 디버그 및 실행하기 위한 엔드투엔드 머신러닝 파이프라인을 더 쉽게 구축할 수 있도록 지원한다'고 말했다.
[{inference_kobart.py:53} INFO - summary_outputs: AWS가 10일 AWS 리인벤트를 통해 머신러닝 서비스인 아마존 세이지메이커 데이터 랭글러를 비롯한 아마존 세이지메이커 피처 스토어 등 9가지 새로운 기능을 발표하며 개발자가 더 나은 가시성, 설명 가능성 및 자동화를 대규모로 구현하는 맞춤형 머신러닝 모델을 준비, 제작, 훈련, 설명, 검사, 모니터링, 디버그 및 실행하기 위한 엔드투엔드 머신러닝 파이프라인을 더 쉽게 구축할 수 있도록 지원한다고 말했다.
[{inference_kobart.py:36} INFO - input text: 대한항공은 이와 같은 필요성에 따라 AWS와 AWS의 국내 파트너사인 LG CNS와 함께 기존 사내 데이터 센터에서 운영했던 데이터와 네트워크, 보안 시스템을 비롯한 각종 IT시스템을 단계적으로 AWS의 클라우드로 이전해 효율성을 높이고 IT 관리를 단순화했다. 대한항공은 이번 전사 IT시스

결괏값들을 확인했다면 로컬 모드로 빠르게 배포하여 테스트하는 것을 권장드립니다. 단, SageMaker Studio는 로컬 모드를 지원하지 않기 때문에 아래 섹션은 SageMaker에서 실행해 주세요.

<br>

## 2. (SageMaker Only) Local Mode Deployment for Model A
---

### Deploy Model A

In [ ]:
modelA_artifact_name = 'modelA.tar.gz'
prepare_model_artifact(modelA_path, model_artifact_name=modelA_artifact_name)
local_model_path = f'file://{os.getcwd()}/{modelA_artifact_name}'

model = PyTorchModel(
    model_data=local_model_path,
    role=role,
    entry_point='inference_nsmc.py', 
    source_dir='src',
    framework_version='1.7.1',
    py_version='py3',
    predictor_cls=NLPPredictor,
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='local'
)

### Invoke using SageMaker Python SDK
SageMaker SDK `predict()` 메서드로 간단하게 추론을 실행할 수 있습니다. 

In [ ]:
inputs = [{"text": ["이 영화는 최고의 영화입니다"]}, 
          {"text": ["최악이에요. 배우의 연기력도 좋지 않고 내용도 너무 허접합니다"]}]

predicted_classes = predictor.predict(inputs)

In [ ]:
for c in predicted_classes:
    print(c)

### Invoke using Boto3 API
이번에는 boto3의 `invoke_endpoint()` 메서드로 추론을 수행해 보겠습니다.
Boto3는 서비스 레벨의 low-level SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 high-level SDK인 SageMaker SDK와 달리 SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다. 

In [ ]:
local_sm_runtime = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = model.endpoint_name

response = local_sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/jsonlines',
    Accept='application/jsonlines',
    Body=modelA_input_data
    )
outputs = response['Body'].read().decode()               

In [ ]:
print_outputs(outputs) 

### Local Mode Endpoint Clean-up
엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.
참고로, 노트북 인스턴스에서 추론 컨테이너를 배포했기 때문에 엔드포인트를 띄워 놓아도 별도로 추가 요금이 과금되지는 않습니다.

로컬 엔드포인트는 도커 컨테이너이기 때문에 `docker rm $(docker ps -a -q)` 으로도 간단히 삭제할 수 있습니다.

In [ ]:
predictor.delete_endpoint()

<br>

## 3. (SageMaker Only) Local Mode Deployment for Model B
---

### Deploy Model B

In [ ]:
modelB_artifact_name = 'modelB.tar.gz'
prepare_model_artifact(modelB_path, model_artifact_name=modelB_artifact_name)
local_model_path = f'file://{os.getcwd()}/{modelB_artifact_name}'

model = PyTorchModel(
    model_data=local_model_path,
    role=role,
    entry_point='inference_korsts.py', 
    source_dir='src',
    framework_version='1.7.1',
    py_version='py3',
    predictor_cls=NLPPredictor,
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='local'
)

### Invoke using SageMaker Python SDK
SageMaker SDK `predict()` 메서드로 간단하게 추론을 실행할 수 있습니다. 

In [ ]:
inputs = [{"text": ["맛있는 라면을 먹고 싶어요", "후루룩 쩝쩝 후루룩 쩝쩝 맛좋은 라면"]}, 
          {"text": ["뽀로로는 내친구", "머신러닝은 러닝머신이 아닙니다."]}]

predicted_classes = predictor.predict(inputs)

In [ ]:
for c in predicted_classes:
    print(c)

### Invoke using Boto3 API
이번에는 boto3의 `invoke_endpoint()` 메서드로 추론을 수행해 보겠습니다.
Boto3는 서비스 레벨의 low-level SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 high-level SDK인 SageMaker SDK와 달리 SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다. 

In [ ]:
local_sm_runtime = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = model.endpoint_name

response = local_sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/jsonlines',
    Accept='application/jsonlines',
    Body=modelB_input_data
    )
outputs = response['Body'].read().decode()               

In [ ]:
print_outputs(outputs)

### Local Mode Endpoint Clean-up
엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.
참고로, 노트북 인스턴스에서 추론 컨테이너를 배포했기 때문에 엔드포인트를 띄워 놓아도 별도로 추가 요금이 과금되지는 않습니다.

로컬 엔드포인트는 도커 컨테이너이기 때문에 `docker rm $(docker ps -a -q)` 으로도 간단히 삭제할 수 있습니다.

In [ ]:
predictor.delete_endpoint()

<br>

## 4. (SageMaker Only) Local Mode Deployment for Model C
---

### Deploy Model C

In [8]:
modelC_artifact_name = 'modelC.tar.gz'
prepare_model_artifact(modelC_path, model_artifact_name=modelC_artifact_name)
local_model_path = f'file://{os.getcwd()}/{modelC_artifact_name}'

model = PyTorchModel(
    model_data=local_model_path,
    role=role,
    entry_point='inference_kobart.py', 
    source_dir='src',
    framework_version='1.7.1',
    py_version='py3',
    predictor_cls=NLPPredictor,
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='local'
)

cp: omitting directory ‘./src/__pycache__’


./
./model.pth
./code/
./code/inference_kobart.py
./code/utils.py
./code/inference_nsmc.py
./code/requirements.txt
./code/inference_korsts.py
Archived modelC.tar.gz
[{session.py:2706} INFO - Creating model with name: pytorch-inference-2022-01-20-09-09-04-955
[{session.py:3066} INFO - Creating endpoint with name pytorch-inference-2022-01-20-09-09-04-956
[{image.py:268} INFO - serving
[{image.py:271} INFO - creating hosting dir in /tmp/tmpqyo55rd6
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[{image.py:1011} INFO - No AWS credentials found in session but credentials from EC2 Metadata Service are available.
[{image.py:683} INFO - docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:

In [10]:
import time
time.sleep(3)

775yc8w3lr-algo-1-uwzc5 | 2022-01-20 09:09:28,328 [INFO ] W-9003-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 12501
775yc8w3lr-algo-1-uwzc5 | 2022-01-20 09:09:28,328 [INFO ] W-9002-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 12504
775yc8w3lr-algo-1-uwzc5 | 2022-01-20 09:09:28,328 [INFO ] W-9000-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 12498
775yc8w3lr-algo-1-uwzc5 | 2022-01-20 09:09:28,337 [INFO ] W-9001-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 12515
775yc8w3lr-algo-1-uwzc5 | 2022-01-20 09:09:28,338 [INFO ] W-9003-model_1 TS_METRICS - W-9003-model_1.ms:12886|#Level:Host|#hostname:bed9bb5e06f2,timestamp:1642669768
775yc8w3lr-algo-1-uwzc5 | 2022-01-20 09:09:28,338 [INFO ] W-9000-model_1 TS_METRICS - W-9000-model_1.ms:12889|#Level:Host|#hostname:bed9bb5e06f2,timestamp:1642669768
775yc8w3lr-algo-1-uwzc5 | 2022-01-20 09:09:28,338 [INFO ] W-9001-model_1 TS_METRICS - W-9001-model_1.ms:12886|#L

### Invoke using Boto3 API
**[주의]** BART 모델은 Auto-Regressive 모델로 내부적으로 연산을 많이 수행하여 기본 인스턴스(예: `ml.t2.medium`)를 사용하는 경우, 시간이 상대적으로 오래 소요됩니다.

In [11]:
local_sm_runtime = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = model.endpoint_name

response = local_sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/jsonlines',
    Accept='application/jsonlines',
    Body=modelC_input_data
    )
outputs = response['Body'].read().decode()             

775yc8w3lr-algo-1-uwzc5 | 2022-01-20 09:09:31,542 [INFO ] W-9000-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle - input text: AWS가 10일 AWS 리인벤트를 통해 머신러닝 서비스인 아마존 세이지메이커(Amazon SageMaker)의 9가지 새로운 기능을 발표했다. 아마존 세이지메이커 데이터 랭글러(Amazon SageMaker Data Wrangler)를 비롯해 아마존 세이지메이커 피처 스토어 등 다양한 기술들이 속속 베일을 벗었다. 스와미 시바수브라마니안(Swami Sivasubramanian), AWS 아마존 머신러닝 부사장은 '수십만 명의 일반 개발자와 데이터 과학자가 업계 최고의 머신러닝 서비스인 아마존 세이지메이커를 활용해 맞춤형 머신러닝 모델 제작, 훈련 및 배치에 대한 장벽을 제거했다'면서 '개발자가 더 나은 가시성, 설명 가능성 및 자동화를 대규모로 구현하는 맞춤형 머신러닝 모델을 준비, 제작, 훈련, 설명, 검사, 모니터링, 디버그 및 실행하기 위한 엔드투엔드 머신러닝 파이프라인을 더 쉽게 구축할 수 있도록 지원한다'고 말했다.
775yc8w3lr-algo-1-uwzc5 | 2022-01-20 09:09:35,800 [INFO ] W-9000-model_1-stdout org.pytorch.serve.wlm.WorkerLifeCycle - summary_outputs: AWS가 10일 AWS 리인벤트를 통해 머신러닝 서비스인 아마존 세이지메이커 데이터 랭글러를 비롯한 아마존 세이지메이커 피처 스토어 등 9가지 새로운 기능을 발표하며 개발자가 더 나은 가시성, 설명 가능성 및 자동화를 대규모로 구현하는 맞춤형 머신러닝 모델을 준비, 제작, 훈련, 설명, 검사, 모니터링, 디버그 및 실행하기 위한 엔드투엔드 머신러닝 파이프라인을 더 쉽게 구축할 수 있도록 지원한다고 말했다.
775yc8w3lr-algo-1-uwzc5 | 2022-01-

In [ ]:
print_outputs(outputs)

### Local Mode Endpoint Clean-up
엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.
참고로, 노트북 인스턴스에서 추론 컨테이너를 배포했기 때문에 엔드포인트를 띄워 놓아도 별도로 추가 요금이 과금되지는 않습니다.

로컬 엔드포인트는 도커 컨테이너이기 때문에 `docker rm $(docker ps -a -q)` 으로도 간단히 삭제할 수 있습니다.

In [ ]:
predictor.delete_endpoint()